In [1]:
import pandas as pd
from pathlib import Path
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import sys, os
import plotly.graph_objects as go


In [4]:
clean_data = os.path.abspath('/Users/macpro/Documents/GitHub/fuel-electric-hybrid-vehicle-ml/data/predicted-data')


In [5]:
filename_2022_1995 = 'vehicle_data_with_clusters.csv'
master_df = pd.read_csv(Path(clean_data,f'{filename_2022_1995}'))


In [14]:
master_df

,vehicle_id,vehicleclass_,make_,model.1_,model_year,cylinders_,fuelconsumption_city(l/100km),fuelconsumption_hwy(l/100km),fuelconsumption_comb(l/100km),co2emissions_(g/km),...,consumption_city(kwh/100km),fuelconsumption_hwy(kwh/100km),fuelconsumption_comb(kwh/100km),fuelconsumption_city(le/100km),fuelconsumption_hwy(le/100km),fuelconsumption_comb(le/100km),range_(km),hybrid_in_fuel,hybrid_in_electric,aggregate_levels
0,fuel-only_1,full-size,acura,integra,2023,4.0,7.9,6.3,7.2,167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
1,fuel-only_2,full-size,acura,integra a-spec,2023,4.0,8.1,6.5,7.4,172,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
2,fuel-only_3,full-size,acura,integra a-spec,2023,4.0,8.9,6.5,7.8,181,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
3,fuel-only_4,suv - small,acura,mdx sh-awd,2023,6.0,12.6,9.4,11.2,263,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
4,fuel-only_5,suv - standard,acura,mdx sh-awd type s,2023,6.0,13.8,11.2,12.4,291,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,electric_420,suv - small,volkswagen,id.4,2023,0.0,0.0,0.0,0.0,0,...,18.2,21.2,19.6,2.0,2.4,2.2,336.0,1,0,0
13092,electric_421,suv - small,volkswagen,id.4 pro,2023,0.0,0.0,0.0,0.0,0,...,18.2,21.3,19.6,2.0,2.4,2.2,443.0,1,0,0
13093,electric_422,suv - small,volkswagen,id.4 awd pro,2023,0.0,0.0,0.0,0.0,0,...,20.1,22.6,21.2,2.3,2.5,2.4,410.0,1,0,0
13094,electric_423,suv - small,volvo,c40 recharge twin,2023,0.0,0.0,0.0,0.0,0,...,22.2,26.1,23.9,2.5,2.9,2.7,364.0,1,0,0


In [23]:
MAKES = master_df['make_'].unique()

MAKES

array(['acura', 'alfa romeo', 'aston martin', 'audi', 'bentley', 'bmw',
       'bugatti', 'buick', 'cadillac', 'chevrolet', 'chrysler', 'dodge',
       'fiat', 'ford', 'genesis', 'gmc', 'honda', 'hyundai', 'infiniti',
       'jaguar', 'jeep', 'kia', 'lamborghini', 'land rover', 'lexus',
       'lincoln', 'maserati', 'mazda', 'mercedes-benz', 'mini',
       'mitsubishi', 'nissan', 'porsche', 'ram', 'rolls-royce', 'subaru',
       'toyota', 'volkswagen', 'volvo', 'smart', 'scion', 'suzuki', 'srt',
       'karma', 'polestar', 'tesla', 'smart eq', 'lucid', 'rivian'],
      dtype=object)

In [49]:
value = 'acura'
filtered_df = master_df[master_df['make_'] == value]


viz_table = pd.DataFrame(filtered_df.groupby(["make_",'model_year'])['predicted_co2_rating'].mean()).reset_index().rename(columns={'predicted_co2_rating':'avg_predicted_co2_rating_by_make'})

px.line(viz_table, x='model_year', y='avg_predicted_co2_rating_by_make', title=f'Average Predicted CO2 Rating by Make ({value.upper()})')

In [50]:
filtered_df = master_df[master_df['make_'] == 'acura']

# create line chart
line_fig = px.scatter(filtered_df, 
                      x='model_year', 
                      y='predicted_co2_rating', 
                      title='Predicted CO2 ratings over time', 
                      labels={'model_year':'Model Year', 'co2emissions_(g/km)':'CO2 Emissions (g/km)'}, 
                      hover_name='model.1_',
                      color='vehicle_type')

line_fig.show()